# System Design: HIPAA-Compliant Medical AI

## 1. Regulatory & Privacy Constraints
- **PII (Personally Identifiable Information)** must be stripped before model inference.
- **Audit Trails**: Every decision must be logged.
- **Explainability**: Black-box models are risky.

## 2. Architecture: Privacy Sidecar Pattern
Incoming Data -> [PII Stripper] -> [Anonymized Data] -> [Model] -> [Prediction] -> [Re-Identification (Only for Authorized UI)]


In [ ]:
import re
from typing import Dict, Any
from datetime import datetime

# --- Component 1: PII Filter ---
class PIIFilter:
    def __init__(self):
        self.patterns = {
            "ssn": r"\d{3}-\d{2}-\d{4}",
            "email": r"[\w\.-]+@[\w\.-]+"
        }
    
    def sanitize(self, text: str) -> str:
        for p_type, pattern in self.patterns.items():
            text = re.sub(pattern, f"[{p_type.upper()}_REDACTED]", text)
        return text

# --- Component 2: Audit Logger ---
class AuditLogger:
    def __init__(self, log_file="audit.log"):
        self.log_file = log_file
    
    def log(self, action: str, details: Dict[str, Any]):
        entry = f"{datetime.now().isoformat()} | {action} | {details}"
        print(f"LOGGED: {entry}")

# --- Workflow ---
pii_filter = PIIFilter()
logger = AuditLogger()

def process_medical_record(raw_record: Dict[str, str]):
    # 1. Sanitize
    notes = raw_record.get("notes", "")
    clean_notes = pii_filter.sanitize(notes)
    
    # 2. Log Access
    logger.log("DATA_ACCESS", {"record_id": raw_record.get("id"), "sanitized": True})
    
    # 3. Model Inference (Simulated)
    # model.predict(clean_notes)
    diagnosis = "Healthy" if "symptoms" not in clean_notes else "Follow-up needed"
    
    return {
        "id": raw_record.get("id"),
        "diagnosis": diagnosis,
        "notes_processed": clean_notes
    }

# Simulation
record = {
    "id": "REC-1001",
    "notes": "Patient John Doe (email: john@example.com) presents with fever."
}

result = process_medical_record(record)
print("\nResult:", result)